In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data preparation

In [ ]:
# import dataset
sales = pd.read_excel('../input/monthly-car-sales-in-quebec-1960/monthly-car-sales.xlsx')
sales.head()

In [ ]:
sales.shape

In [ ]:
sales.tail()

In [ ]:
#  Check stationarity
sales.iloc[:,1].plot()
plt.show()

In [ ]:
# Преобразуем данные
sales_2 = pd.DataFrame()

for i in range(12,0,-1):
    sales_2['t-'+str(i)] = sales.iloc[:,1].shift(i)

sales_2['t'] = sales.iloc[:,1].values

print(sales_2.head(23))

In [ ]:
# Преобразуем данные
sales_2 = pd.DataFrame()

for i in range(12,0,-1):
    sales_2['t-'+str(i)] = sales.iloc[:,1].shift(i)

sales_2['t'] = sales.iloc[:,1].values

print(sales_2.head(23))

In [ ]:
#  Отрезаем первые 12 строк
sales_4 = sales_2[12:]

sales_4.head()

In [ ]:
#  предикторы и отклик разделяем
# Отклик - группирующая переменная -  вектор y
y = sales_4['t']
# Предикторы - таблица X
X = sales_4.drop('t', axis=1)

In [ ]:
#  Split on train and validation sets
X_train = X[:91]
y_train = y[:91]
X_test  = X[91:]
y_test  = y[91:]

In [ ]:
print(sales_4.shape)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(X_train.head(5))

# Prediction using neural networks
## Default model
No gap between train and test data.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Creating a model
model = Sequential()
# внутренний слой с 8 нейронами, 12 столбцов 
model.add(Dense(8, input_dim=12, activation='relu'))
# выходной слой с линейной комбинацией. Почему не логистическая ?
model.add(Dense(1, activation='linear'))


# Compiling model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_error', 'mean_absolute_percentage_error'])


# Training a model
model.fit(X_train, y_train, epochs=300, batch_size=None, verbose=0)

In [ ]:
# оценка качества модели на тестовом множестве
# что здесь не так?
scores = model.evaluate(X_test, y_test)
# print("\nMAPE: %.2f%%" % (scores[1]))

### Results
- Blue line - ground truth values
- Green line - predicted values on train data
- Red line - predicted values on test data

In [ ]:
def plot_results(pred_train, pred_test, x_train_plot, x_test_plot, y_train, y_test):
    # Ground-truth values
    plt.plot(x_train_plot, y_train, color='blue')
    plt.plot(x_test_plot, y_test, color='blue')

    # Predicted values
    plt.plot(x_train_plot, pred_train, color='green')
    plt.plot(x_test_plot, pred_test, color='red')
    plt.show()

In [ ]:
# Compute results
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

x_train_plot = np.arange(0, 91, 1)
x_test_plot = np.arange(91, 96, 1)
plot_results(predictions_train, predictions_test, x_train_plot, x_test_plot, y_train, y_test)

## Custom model
In the model, a gap between test and train data is added

### Training

In [ ]:
# Remove intermediate data between train and test sets
X_train_trimmed = X[:80]
y_train_trimmed = y[:80]

x_tr_t_plot = np.arange(0, 80, 1)


plt.plot(x_tr_t_plot, y_train_trimmed, color='blue')
plt.plot(x_test_plot, y_test, color='blue')
plt.show()

In [ ]:
# Training a model
model.fit(X_train_trimmed, y_train_trimmed, epochs=300, batch_size=None, verbose=0)

In [ ]:
# Evaluation
scores = model.evaluate(X_test, y_test)

nn_scores = scores[1:]
overall_results = {'Neural Network': nn_scores}
print(overall_results)

### Results
- Blue line - ground truth values
- Green line - predicted values on train data
- Red line - predicted values on test data

In [ ]:
# Compute results
predictions_train = model.predict(X_train_trimmed)
predictions_test = model.predict(X_test)

plot_results(predictions_train, predictions_test, x_tr_t_plot, x_test_plot, y[:80], y[91:])

# Arima
First of all, plot data to analyse its type. To see whether it's staionary or not.

In [ ]:
#  Check stationarity
from statsmodels.tsa.stattools import adfuller
X = sales['Monthly car sales in Quebec 1960-1968']
result = adfuller(X)

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
if (result[1] > 0.05):
    print('Time-series data is not stationary')


## Data preprocessing

In [ ]:
sales = pd.read_excel('../input/monthly-car-sales-in-quebec-1960/monthly-car-sales.xlsx',
                      index_col=0,
                     parse_dates=True)
sales.rename(columns={'Monthly car sales in Quebec 1960-1968': 'Car sales'}, inplace=True)
sales.plot()
plt.show()

## ARIMA parameters
Using auto correlation plot, we can say that a good starting point for the AR parameter of the model may be 38-40.

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(sales)
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

X_train_ar = sales.values[:91]
X_test_ar = sales.values[91:]
# fit model
model = ARIMA(X_train_ar, order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

## Results

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

predictions = list()
history = [x for x in X_train_ar]


for x in range(len(X_test_ar)):
    model = ARIMA(history, order=(5, 1, 0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    
    predictions.append(output[0])
    history.append(X_test_ar[x])
    
arima_mse = mean_squared_error(X_test_ar, predictions)
arima_mae = mean_absolute_error(X_test_ar, predictions)
arima_mape = mean_absolute_percentage_error(X_test_ar, predictions)
overall_results['ARIMA'] = [arima_mae, arima_mse, arima_mape]    
    
plt.plot(X_test_ar, color='blue')
plt.plot(predictions, color='red')
plt.show()

# Decision trees

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# fit an random forest model and make a one step prediction
def random_forest_forecast(Xtrain, ytrain, Xtest, ytest):
    # fit model
    model = RandomForestRegressor(n_estimators=1000)
    model.fit(Xtrain, ytrain)
    # make a one-step prediction
    yhat = model.predict(Xtest)
    
    return yhat[0]


# walk-forward validation for univariate data
def walk_forward_validation(Xtrain, ytrain, Xtest, ytest):
    predictions = list()
    
    # seed history with training dataset
    x_history = [x for x in Xtrain]
    y_history = [y for y in ytrain]
    # step over each time-step in the test set
    for i in range(len(Xtest)):
        # fit model on history and make a prediction
        yhat = random_forest_forecast(x_history, y_history, Xtest, ytest)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        x_history.append(Xtest[i])
        y_history.append(ytest[i])
    
    return predictions

X_train_dt = np.asarray(X_train)
y_train_dt = np.asarray(y_train)
X_test_dt = np.asarray(X_test)
y_test_dt = np.asarray(y_test)
predictions = walk_forward_validation(X_train_dt, y_train_dt, X_test_dt, y_test_dt)

# estimate prediction error
dt_mae = mean_absolute_error(y_test_dt, predictions)
dt_mse = mean_squared_error(y_test_dt, predictions)
dt_mape = mean_absolute_percentage_error(y_test_dt, predictions)
overall_results['Decision trees'] = [dt_mae, dt_mse, dt_mape]

## Results

In [ ]:
plt.plot(y_test_dt, color='blue')
plt.plot(predictions, color='red')
plt.show()

# Overall results

In [ ]:
df = pd.DataFrame.from_dict(overall_results, orient='index', columns=['MAE', 'MSE', 'MAPe'])
df.transpose()
df.round(6)